In [8]:
from __future__ import print_function
import keras
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda
from siamese import SiameseNetwork
import numpy as np

In [2]:
batch_size = 128
num_classes = 10
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [27]:
def create_base_model(input_shape):
    model_input = Input(shape=input_shape)

    embedding = Conv2D(32, kernel_size=(3, 3), input_shape=input_shape)(model_input)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Conv2D(64, kernel_size=(3, 3))(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Flatten()(embedding)
    embedding = Dense(128)(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)

    return Model(model_input, embedding)


def create_head_model(embedding_shape):
    embedding_a = Input(shape=embedding_shape[1:])
    embedding_b = Input(shape=embedding_shape[1:])

#     head = Concatenate()([embedding_a, embedding_b])
#     head = Dense(8)(head)
#     head = BatchNormalization()(head)
#     head = Activation(activation='sigmoid')(head)

#     head = Dense(1)(head)
#     head = BatchNormalization()(head)
#     head = Activation(activation='sigmoid')(head)
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([embedding_a, embedding_b])
    prediction = Dense(1,activation='sigmoid')(L1_distance)

    return Model([embedding_a, embedding_b], prediction)

In [9]:
base_model = create_base_model(input_shape)
head_model = create_head_model(base_model.output_shape)

siamese_network = SiameseNetwork(base_model, head_model)
siamese_network.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

siamese_checkpoint_path = "./siamese_checkpoint"

siamese_callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, verbose=0),
    ModelCheckpoint(siamese_checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=0)
]

In [10]:
siamese_network.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    batch_size=1000,
                    epochs=epochs,
                    callbacks=siamese_callbacks)

C:\Users\cherry\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
60/60 [==============================] - 124s 2s/step - loss: 0.3378 - accuracy: 0.8738 - val_loss: 0.6303 - val_accuracy: 0.8505
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 2/10
60/60 [==============================] - 134s 2s/step - loss: 0.1729 - accuracy: 0.9554 - val_loss: 0.5632 - val_accuracy: 0.7800
Epoch 3/10
60/60 [==============================] - 112s 2s/step - loss: 0.1137 - accuracy: 0.9721 - val_loss: 0.3731 - val_accuracy: 0.9042
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 4/10
60/60 [==============================] - 119s 2s/step - loss: 0.0843 - accuracy: 0.9787 - val_loss: 0.1846 - val_accuracy: 0.9575
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 5/10
60/60 [==============================] - 137s 2s/step - loss: 0.0696 - accuracy: 0.9819 - val_loss: 0.1009 - val_accuracy: 0.9727
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 6/10
60/60 [=========================

In [11]:
siamese_network.evaluate(x_train, y_train, batch_size=64)

C:\Users\cherry\anaconda3\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.03771126642823219, 0.9888559579849243]

In [22]:
base_model = create_base_model(input_shape)
head_model = create_head_model(base_model.output_shape)

siamese_network = SiameseNetwork(base_model, head_model)
siamese_network.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

siamese_checkpoint_path = "./siamese_checkpoint"

siamese_callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, verbose=0),
    ModelCheckpoint(siamese_checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=0)
]

few_shot_index = [np.where(y_train == i)[0][:20] for i in range(num_classes)]
few_shot_index = np.concatenate(few_shot_index, axis=None)
x_train_fewshot = x_train[few_shot_index]
y_train_fewshot = y_train[few_shot_index]

siamese_network.fit(x_train_fewshot, y_train_fewshot,
                    validation_data=(x_test[:500], y_test[:500]),
                    batch_size=10,
                    epochs=20,
                    callbacks=siamese_callbacks)

C:\Users\cherry\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
20/20 [==============================] - 3s 70ms/step - loss: 0.6831 - accuracy: 0.5750 - val_loss: 0.6860 - val_accuracy: 0.5320
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 2/20
20/20 [==============================] - 1s 54ms/step - loss: 0.6169 - accuracy: 0.7050 - val_loss: 0.6825 - val_accuracy: 0.5460
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 3/20
20/20 [==============================] - 1s 58ms/step - loss: 0.5979 - accuracy: 0.6950 - val_loss: 0.6807 - val_accuracy: 0.6100
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 4/20
20/20 [==============================] - 1s 56ms/step - loss: 0.5293 - accuracy: 0.7950 - val_loss: 0.6745 - val_accuracy: 0.6760
INFO:tensorflow:Assets written to: .\siamese_checkpoint\assets
Epoch 5/20
20/20 [==============================] - 1s 52ms/step - loss: 0.5299 - accuracy: 0.8250 - val_loss: 0.6667 - val_accuracy: 0.7040
INFO:tensorflow:Assets written to: .\siames

In [23]:
siamese_network.evaluate(x_test, y_test, batch_size=10)

C:\Users\cherry\anaconda3\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.41422051191329956, 0.8327999711036682]